<a href="https://colab.research.google.com/github/achanhon/coursdeeplearningcolab/blob/master/scikitTOpytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# scikit-learn vers pytorch

L'objet de ce petit notebook est de montrer comment on peut implémenter les MLP de scikit-learn en pytorch (sachant que pytorch permet d'utiliser n'importe quel type de réseau de neurones et pas nécessairement des MLP).

Normalement, les différences de comportement sont uniquements du à l'aléatoire car la réimplémentation pytorch est basé sur la code source du MLP de scikit-learning https://github.com/scikit-learn/scikit-learn/tree/main/sklearn/neural_network .




##MNIST
Je propose d'utiliser le jeu de données MNIST pour effectuer la comparaison.

Pour cela, récupérons le dataset à l'aide de torchvision, puis convertissons le en format vectoriel pour scikit-learn.

(Notons, que le dataset torchvision peut directement être donnée à un dataloader torchvision qui fait des paquets - ce qui est pertinent vu que l'optimisation n'est pas globale - cependant c'est le format scikit qui veut ça.)

In [6]:
import numpy
import torch
import torchvision
import sklearn

mnisttrain = torchvision.datasets.MNIST("./mnist",train=True, transform=torchvision.transforms.ToTensor(), download=True)
mnisttest = torchvision.datasets.MNIST("./mnist",train=False, transform=torchvision.transforms.ToTensor(), download=True)

Xtrain,Ytrain = torch.zeros(len(mnisttrain),28,28),torch.zeros(len(mnisttrain))
Xtest,Ytest = torch.zeros(len(mnisttest),28,28),torch.zeros(len(mnisttest))

for i in range(Xtrain.shape[0]):
  Xtrain[i],Ytrain[i] = mnisttrain[i]
for i in range(Xtest.shape[0]):
  Xtest[i],Ytest[i] = mnisttest[i]

Xtrain,Xtest = Xtrain.flatten(1),Xtest.flatten(1)


##MNIST avec scikit-learn

In [7]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(Xtrain,Ytrain)

MLPClassifier(max_iter=300, random_state=1)

In [8]:
clf.score(Xtest.numpy(),Ytest.numpy())

0.9792

## Même chose avec pytorch


In [9]:
class MLP(torch.nn.Module):
  def __init__(self,hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, max_fun=15000):
    super(MLP, self).__init__()
    self.hidden_layer_sizes = hidden_layer_sizes
    self.activation = activation
    self.solver = solver
    self.alpha = alpha
    self.batch_size = 'auto'
    self.learning_rate = learning_rate
    self.learning_rate_init=learning_rate_init
    self.power_t=power_t
    self.max_iter=max_iter
    self.shuffle=shuffle
    self.random_state=random_state
    self.tol=tol
    self.verbose=verbose
    self.warm_start=warm_start
    self.momentum=momentum
    self.nesterovs_momentum=nesterovs_momentum
    self.early_stopping=early_stopping
    self.validation_fraction=validation_fraction
    self.beta_1=beta_1
    self.beta_2=beta_2
    self.epsilon=epsilon
    self.n_iter_no_change=n_iter_no_change
    self.max_fun=max_fun

    if activation!='relu' or solver!='adam':
      print("ben faut le faire XD")
      quit()

  def forward(self,x):
    for i in range(len(self.linears)):
      x = self.linears[i](x)
      x = torch.nn.functional.leaky_relu(x)
    return x

  def fit(self,X,Y):
    nbclass = torch.max(Y)
    dim = X.shape[1]

    self.linears = torch.nn.ModuleList([torch.nn.Linear(dim, self.hidden_layer_sizes[0])])
    for i in range(1,len(self.hidden_layer_sizes[0])):
      self.linears.append(torch.nn.Linear(self.hidden_layer_sizes[i-1], self.hidden_layer_sizes[i]))
    self.linears.append(torch.nn.Linear(self.hidden_layer_sizes[-1], nbclass))

    self.solver =torch.optim.Adam(self.parameters(), lr=self.learning_rate_init)
    self.criterion = torch.nn.CrossEntropyLoss()

    dataset = torch.utils.data.TensorDataset(X,Y)
    if isinstance(self.batch_size, str):
      self.batch_size = 128 # ;-)

    dataloader = torch.utils.data.DataLoader(dataset,batch_size=64, shuffle=True)

  def predict(self,X):

  def score(self,X,Y):


SyntaxError: ignored